Load data from dataset and preprocess it

In [ ]:
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pandas.io.json import json_normalize
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

#read train.json data
orign_train_data = pd.read_json('train.json')

#deep copy the original data
train_data = copy.deepcopy(orign_train_data)

#Get the authors list
train_data_authors = train_data['authors']

prolific_authors = []
coauthors = []
#Get the prolific authors list to train the model by removing the coauthors
for author in train_data_authors:
    p_authors = []
    np_authors = []
    for name in author:
        if name < 100:
            p_authors.append(name)
        else:
            np_authors.append(name)
    prolific_authors.append(p_authors)
    coauthors.append(np_authors)

#add the prolific authors list to the train data
train_data['coauthors'] = coauthors
train_data['prolific_authors'] = prolific_authors

#remove authors in the train data
train_data = train_data.drop(['authors'], axis=1)

p_a = train_data['prolific_authors']

#read test.json data
test_data = pd.read_json('test.json')

#Pack the prediction result into a csv file
def pack_result(result, file_name):
    result = pd.DataFrame(result)
    #change the 'ID' and 'Predict' column to int32
    result['ID'] = result['ID'].astype('int32')
    result['Predict'] = result['Predict'].astype('int32')
    result.columns = ['ID', 'Predict']
    result.to_csv(file_name, index=False)





Split the data into training and testing sets and make the no prolific paper has the -1 label

In [172]:

#Split the train data into training set and validation set
X_train, X_test, y_train, y_test = train_test_split(train_data.drop(['prolific_authors'],axis=1), p_a, test_size=0.2, random_state=42)

#get the same index entries in the original train data for training the model
X_train_orgin = orign_train_data.loc[X_train.index]

#re order the index of the X_train_orgin
X_train_orgin = X_train_orgin.reset_index(drop=True)

#re order the index of the y_train
y_train = y_train.reset_index(drop=True)

#re order the index of the X_test
X_test = X_test.reset_index(drop=True)

#re order the index of the y_test
y_test = y_test.reset_index(drop=True)

for i in range(len(y_train)):
    # if the author list is empty, add -1 to the list
    if len(y_train.iloc[i]) == 0:
        y_train.iloc[i].append(-1)

for i in range(len(y_test)):
    # if the author list is empty, add -1 to the list
    if len(y_test.iloc[i]) == 0:
        y_test.iloc[i].append(-1)



Functions that used to find information from the dataset

In [173]:
all_authors = X_train_orgin['authors']

#find authors that author x used to worked with and how many times
def find_author_x_work_with(author):
    author_x_work_with = {}
    author_x_paper = find_author_x_paper(author)
    for paper in author_x_paper:
        for ath in all_authors.loc[paper]:
            if ath != author:
                if ath in author_x_work_with:
                    author_x_work_with[ath] += 1
                else:
                    author_x_work_with[ath] = 1
    return author_x_work_with

#find which paper that author x participated
def find_author_x_paper(author):
    author_x_paper = []
    for i in range(len(all_authors)):
        if author in all_authors.loc[i]:
            author_x_paper.append(i)
    return author_x_paper

#find the venue that author published papers
def find_author_venue(author):
    venues = {}
    author_paper = find_author_x_paper(author)
    for paper in author_paper:
        venue = X_train_orgin.loc[paper]['venue']
        if venue in venues:
            venues[venue] += 1
        else:
            venues[venue] = 1
    return venues


#whether the author x and author y worked with each other
def is_author_x_work_with_author_y(author_x, author_y):
    author_x_paper = find_author_x_paper(author_x)
    author_y_paper = find_author_x_paper(author_y)
    for paper in author_x_paper:
        if paper in author_y_paper:
            return True
    return False
    
#find the prolific authors that the authors worked with (at least one of given authors worked with)
def get_authors_prolific_atleast_one(authors):
    prolific_authors = []
    for author in authors:
        prolific_authors.append(find_author_x_work_with(author))
    #get the prolific authors that these authors all worked with
    prolific_authors = set.union(*map(set, prolific_authors))

    p_a = []
    for author in prolific_authors:
        if author < 100:
            p_a.append(author)
    return p_a
           

Feature engineering process for the logistic regression model training

In [184]:
from sklearn.linear_model import LogisticRegression

def feature_engineering(train,test):
    new_featurs = {}

    #find the ratio of how many coauthors had worked with the prolific author
    new_featurs['work_ratio'] = []

    #see if the time of prolific author has worked with the author in the same venue
    new_featurs['same_venue'] = []

    #how many coauthors in the given data
    new_featurs['coauthors_count'] = []

    #how mant prolific authors that at least one of the coauthors has worked with
    new_featurs['prolific_count'] = []

    #whether this prolific author has participated in the same paper with the given coauthors
    new_featurs['is_p_author'] = []

    for i in range(len(train)):
        authors = train.iloc[i]['coauthors']

        #the paper that have only one author and it is also a prolific author will have no coauthors
        if len(authors) != 0:

            #create two dictionary to store the prolific authors and paper that the prolific authors have participated
            worked_authors_dic = {}
            published_paper_dic = {}
            true_lable = test.iloc[i]

            all_prolific_authors = get_authors_prolific_atleast_one(authors)

            #these two features are same for all the possible prolific authors in the given paper
            prolific_count = len(all_prolific_authors)
            coauthors_count = len(authors)

            if all_prolific_authors != []:
                for p_author in all_prolific_authors:
                    
                    all_p_authors_worked_with = find_author_x_work_with(p_author)
                    #copy the keys of the all_p_authors_worked_with
                    all_p_authors_worked_with_keys = copy.deepcopy(list(all_p_authors_worked_with.keys()))

                    for p_w_author in all_p_authors_worked_with_keys:
                        #remove the authors that the author have not worked with
                        if p_w_author not in authors:
                            all_p_authors_worked_with.pop(p_w_author)

                    worked_authors_dic[p_author] = all_p_authors_worked_with
            
            for wad in worked_authors_dic:
                work_ratio = len(worked_authors_dic[wad])/coauthors_count
                # print('author', wad, 'ratio', work_ratio)

                # venue that prolific author has published paper
                ven = find_author_venue(wad)

                #count the common venue
                same_count = 0
                for v in ven.keys():
                    paper_venue = train.iloc[i]['venue']
                    if v == paper_venue:
                        same_count += ven[v]

                #add the features to the new_featurs
                new_featurs['work_ratio'].append(work_ratio)
                new_featurs['same_venue'].append(same_count)
                new_featurs['coauthors_count'].append(coauthors_count)
                new_featurs['prolific_count'].append(prolific_count)
                if wad in true_lable:
                    new_featurs['is_p_author'].append(1)
                else:
                    new_featurs['is_p_author'].append(0)

            # print('all_prolific_authors', all_prolific_authors)
            # print('the coauthors of paper', i, 'are', authors)
            # print('worked authors dic', worked_authors_dic)
            
            print('== ',i,'======================='*5)
    
    #return the new features as pandas dataframe
    new_featurs = pd.DataFrame(new_featurs)

    #Save the data into a json file
    new_featurs.to_json('new_featurs.json')

    return new_featurs

#create a logistic regression model
def logistic_regression_model(X_train, y_train, penalty='l2', C=1.0, solver='liblinear', max_iter=100, l1_ratio=None):
    logistic_regression = LogisticRegression()
    logistic_regression.fit(X_train, y_train)
    return logistic_regression

# Determine whether the author is the author of the paper using logistic regression
def is_prolific_author(author,paper_info):
    authors = paper_info['coauthors']
    venue = paper_info['venue']
    all_prolicific_authors = get_authors_prolific_atleast_one(authors)
    if all_prolicific_authors != []:
        for p_author in all_prolicific_authors:
            #get the ratio of how many coauthors had worked with the prolific author

            return 1
    else:
        return -1


# feature_engineering(X_train[0:1000], y_train[0:1000])
# print(X_train_orgin)
print(get_authors_prolific_atleast_one([91]))

[14, 19, 23, 38, 51, 57, 81]


In [179]:
new_featurs = pd.read_json('features_data.json')

#print the first 5 rows of the features
new_featurs.head()

#traing the logistic regression model
traing_features = np.array(new_featurs[['work_ratio','same_venue','coauthors_count','prolific_count']])
traing_labels = np.array(new_featurs['is_p_author'])
LR = logistic_regression_model(traing_features, traing_labels)

#predict the test data
prediction = {}
prediction['ID'] = []
prediction['Predict'] = []

#remove all the rows that have no coauthors
print('the length of the test data before removing the rows that have no coauthors', len(X_test))
X_test = X_test[X_test['coauthors'].map(len) != 0]
#remove the same index in the y_test
y_test = y_test[X_test.index]
print('the length of the test data after removing the rows that have no coauthors', len(X_test))
    
# for i in range(len(X_test[0:10])):
#     authors = X_test.iloc[i]['coauthors']
#     paper_info = X_test.iloc[i]
#     prediction['ID'].append(i)
#     apa = get_authors_prolific_atleast_one(authors)
#     if apa != []:
#         prediction['Predict'].append(1)
#     else:
#         prediction['Predict'].append(-1)

#save the prediction into a csv file
# prediction = pd.DataFrame(prediction)
# prediction.to_csv('prediction.csv', index=False)



the length of the test data before removing the rows that have no coauthors 5079
the length of the test data after removing the rows that have no coauthors 5079


In [187]:
print(X_test[0:20])
print(y_test[0:20])

print(get_authors_prolific_atleast_one([4286, 10812]))
#要解决的问题，由于X_train被重新排序，所以对测试数据进行预测的时候，需要原来的index，这是由于关键方程 get_author_paper 的原理导致的


    year                                           abstract venue  \
0      7  [1623, 1621, 1700, 1543, 33, 2155, 1669, 1916,...   129   
1     10  [40, 1542, 1691, 2380, 1604, 1839, 1553, 1535,...    44   
2      4  [46, 1605, 1691, 1557, 2227, 1535, 6, 2036, 7,...   195   
3     10  [3892, 1803, 1532, 11, 1650, 2414, 1692, 1866,...    44   
4     17  [1529, 2264, 1542, 1529, 3204, 3448, 45, 1559,...     9   
6     11  [1532, 2783, 1905, 1613, 3743, 1535, 2079, 232...     5   
7     13  [40, 1542, 1691, 2449, 1535, 2992, 1543, 33, 1...    42   
8      7  [37, 2227, 33, 1642, 1586, 1720, 1703, 1814, 3...    10   
9      5  [37, 2495, 1535, 1616, 3296, 2254, 43, 1553, 3...    32   
10    15  [37, 1558, 1671, 2526, 1594, 46, 3001, 1575, 6...     9   
11    16  [37, 2611, 3298, 2182, 2210, 1525, 1535, 1624,...    28   
12     9  [1731, 2610, 1543, 1535, 37, 34, 16, 12, 14, 1...    53   
13    16  [53, 1546, 1752, 1682, 2565, 1659, 3713, 1543,...    10   
14    16  [37, 2087, 1608, 1772, 1